In [ ]:
# ============================================================
#  FASTING VITALS ANALYZER & PREDICTOR
#  Medical Informatics Portfolio Project
#  Compatible with Google Colab — run as a single cell
# ============================================================

# ── 0. Install / Import Libraries ───────────────────────────
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Fix random seed for reproducibility
np.random.seed(42)
N = 500  # number of synthetic patients


# ============================================================
# STEP 1 — GENERATE SYNTHETIC HEALTH DATA
# ============================================================

# ── 1a. Demographic & baseline features ─────────────────────
patient_id          = np.arange(1, N + 1)
age                 = np.random.randint(18, 66, N)
gender              = np.random.choice(["Male", "Female"], N)
days_fasted         = np.random.randint(5, 31, N)                        # 5–30 days
avg_fasting_hours   = np.round(np.random.uniform(13.0, 16.0, N), 1)      # 13–16 hrs/day
baseline_weight     = np.round(np.random.uniform(60.0, 120.0, N), 1)     # kg
baseline_fbs        = np.round(np.random.uniform(90.0, 140.0, N), 1)     # mg/dL
sleep_quality       = np.random.randint(1, 11, N)                        # 1–10 score

# ── 1b. Realistic weight loss formula ───────────────────────
# Core drivers:
#   • Days fasted     → linear calorie deficit accumulation
#   • Fasting hours   → longer daily window ≈ greater deficit
#   • Baseline weight → heavier individuals lose slightly more
# Formula derived from simplified energy-balance assumptions:
#   ~0.04 kg/day base rate, scaled by fasting hours ratio,
#   with a small boost from higher starting weight.

base_rate       = 0.04                                # kg per day baseline
hour_factor     = (avg_fasting_hours - 13.0) / 3.0   # 0→1 as hours go 13→16
weight_factor   = (baseline_weight - 60.0) / 120.0   # 0→0.5 for 60→120 kg

weight_loss = (
    base_rate * days_fasted * (1 + 0.5 * hour_factor + 0.2 * weight_factor)
    + np.random.normal(0, 0.15, N)   # random physiological noise
)
weight_loss     = np.clip(weight_loss, 0.1, None)     # no negative weight loss
current_weight  = np.round(baseline_weight - weight_loss, 2)
weight_loss     = np.round(weight_loss, 2)

# ── 1c. Realistic blood sugar drop formula ──────────────────
# Fasting typically reduces FBS by ~0.3–0.8 mg/dL per day,
# modulated by baseline level (higher baseline → bigger drop)
# and sleep quality (better sleep → better insulin sensitivity).

bs_daily_drop   = 0.5 + 0.003 * (baseline_fbs - 90.0) + 0.02 * sleep_quality
blood_sugar_drop = (
    bs_daily_drop * days_fasted
    + np.random.normal(0, 1.5, N)
)
blood_sugar_drop    = np.clip(blood_sugar_drop, 0, None)
current_fbs         = np.round(baseline_fbs - blood_sugar_drop, 1)
current_fbs         = np.clip(current_fbs, 70.0, None)   # physiological floor
blood_sugar_drop    = np.round(blood_sugar_drop, 2)

# ── 1d. Assemble DataFrame ──────────────────────────────────
df = pd.DataFrame({
    "Patient_ID"                    : patient_id,
    "Age"                           : age,
    "Gender"                        : gender,
    "Days_Fasted"                   : days_fasted,
    "Avg_Fasting_Hours"             : avg_fasting_hours,
    "Baseline_Weight_kg"            : baseline_weight,
    "Current_Weight_kg"             : current_weight,
    "Weight_Loss_kg"                : weight_loss,
    "Baseline_Fasting_Blood_Sugar"  : baseline_fbs,
    "Current_Fasting_Blood_Sugar"   : current_fbs,
    "Blood_Sugar_Drop"              : blood_sugar_drop,
    "Sleep_Quality_Score"           : sleep_quality,
})

print("═" * 55)
print("  DATASET OVERVIEW")
print("═" * 55)
print(df.describe().round(2).to_string())
print(f"\nShape : {df.shape}")
print(f"Nulls : {df.isnull().sum().sum()}")


# ============================================================
# STEP 2 — MACHINE LEARNING MODEL (Random Forest Regressor)
# ============================================================

FEATURES = ["Age", "Days_Fasted", "Avg_Fasting_Hours", "Baseline_Weight_kg"]
TARGET   = "Weight_Loss_kg"

X = df[FEATURES].values
y = df[TARGET].values

# 80/20 train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Train Random Forest (100 estimators, robust to outliers & nonlinear relations)
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)

# Evaluate
y_pred  = model.predict(X_test)
r2      = r2_score(y_test, y_pred)
mse     = mean_squared_error(y_test, y_pred)
rmse    = np.sqrt(mse)

print("\n" + "═" * 55)
print("  MODEL PERFORMANCE  (Random Forest Regressor)")
print("═" * 55)
print(f"  R² Score (accuracy) : {r2:.4f}")
print(f"  MSE                 : {mse:.4f} kg²")
print(f"  RMSE                : {rmse:.4f} kg")

# Feature importance scores
importances = model.feature_importances_
importance_df = pd.DataFrame({
    "Feature"   : FEATURES,
    "Importance": np.round(importances, 4)
}).sort_values("Importance", ascending=True)

print("\n  Feature Importances:")
for _, row in importance_df.iterrows():
    bar = "█" * int(row["Importance"] * 50)
    print(f"  {row['Feature']:25s} {bar} {row['Importance']:.4f}")


# ============================================================
# STEP 3 — INTERACTIVE PLOTLY DASHBOARD
# ============================================================

# ── Colour palette ───────────────────────────────────────────
DARK_BG     = "#0d1117"
PANEL_BG    = "#161b22"
ACCENT_TEAL = "#00c9a7"
ACCENT_PINK = "#ff6b9d"
ACCENT_GOLD = "#ffd166"
TEXT_COLOR  = "#e6edf3"
GRID_COLOR  = "#21262d"

GENDER_COLORS = {"Male": ACCENT_TEAL, "Female": ACCENT_PINK}

# ── Figure 1: Scatter — Days Fasted vs Weight Loss ──────────
fig1 = px.scatter(
    df,
    x="Days_Fasted",
    y="Weight_Loss_kg",
    color="Gender",
    size="Avg_Fasting_Hours",
    size_max=14,
    color_discrete_map=GENDER_COLORS,
    opacity=0.82,
    hover_data={
        "Patient_ID"        : True,
        "Age"               : True,
        "Days_Fasted"       : True,
        "Avg_Fasting_Hours" : True,
        "Weight_Loss_kg"    : ":.2f",
        "Gender"            : True,
    },
    title="Effect of Fasting Duration on Weight Loss",
    labels={
        "Days_Fasted"   : "Days Fasted",
        "Weight_Loss_kg": "Weight Loss (kg)",
    },
)

# Add a trend-line per gender using numpy polyfit
for g, col in GENDER_COLORS.items():
    sub = df[df["Gender"] == g]
    z   = np.polyfit(sub["Days_Fasted"], sub["Weight_Loss_kg"], 1)
    p   = np.poly1d(z)
    x_line = np.linspace(sub["Days_Fasted"].min(), sub["Days_Fasted"].max(), 100)
    fig1.add_trace(go.Scatter(
        x=x_line, y=p(x_line),
        mode="lines",
        line=dict(color=col, width=2, dash="dash"),
        name=f"{g} trend",
        showlegend=True,
    ))

fig1.update_layout(
    paper_bgcolor=DARK_BG,
    plot_bgcolor=PANEL_BG,
    font=dict(color=TEXT_COLOR, family="'Courier New', monospace"),
    title_font=dict(size=18, color=ACCENT_GOLD),
    xaxis=dict(gridcolor=GRID_COLOR, zerolinecolor=GRID_COLOR),
    yaxis=dict(gridcolor=GRID_COLOR, zerolinecolor=GRID_COLOR),
    legend=dict(bgcolor=PANEL_BG, bordercolor=GRID_COLOR, borderwidth=1),
    margin=dict(t=60, b=50, l=60, r=30),
)

# ── Figure 2: Box Plot — Blood Sugar Before vs After ────────
# Melt to long format for grouped box plot
bs_melt = df[["Baseline_Fasting_Blood_Sugar", "Current_Fasting_Blood_Sugar"]].melt(
    var_name="Measurement",
    value_name="Blood_Sugar_mg_dL"
)
bs_melt["Measurement"] = bs_melt["Measurement"].map({
    "Baseline_Fasting_Blood_Sugar": "Baseline",
    "Current_Fasting_Blood_Sugar" : "After Fasting",
})

fig2 = px.box(
    bs_melt,
    x="Measurement",
    y="Blood_Sugar_mg_dL",
    color="Measurement",
    color_discrete_map={
        "Baseline"     : ACCENT_PINK,
        "After Fasting": ACCENT_TEAL,
    },
    points="all",           # show underlying data points
    notched=True,           # notched box → visual confidence interval
    title="Blood Sugar: Baseline vs Current (After Fasting)",
    labels={
        "Measurement"      : "",
        "Blood_Sugar_mg_dL": "Fasting Blood Sugar (mg/dL)",
    },
)
fig2.update_traces(
    marker=dict(opacity=0.25, size=3),
    jitter=0.35,
)
fig2.update_layout(
    paper_bgcolor=DARK_BG,
    plot_bgcolor=PANEL_BG,
    font=dict(color=TEXT_COLOR, family="'Courier New', monospace"),
    title_font=dict(size=18, color=ACCENT_GOLD),
    xaxis=dict(gridcolor=GRID_COLOR),
    yaxis=dict(gridcolor=GRID_COLOR, zerolinecolor=GRID_COLOR),
    showlegend=False,
    margin=dict(t=60, b=50, l=60, r=30),
)

# ── Figure 3: Horizontal Bar — Feature Importance ───────────
bar_colors = [ACCENT_TEAL, ACCENT_PINK, ACCENT_GOLD, "#a78bfa"]

fig3 = go.Figure(go.Bar(
    x=importance_df["Importance"],
    y=importance_df["Feature"],
    orientation="h",
    marker=dict(
        color=bar_colors[:len(importance_df)],
        line=dict(color=DARK_BG, width=1.5),
    ),
    text=[f"{v:.4f}" for v in importance_df["Importance"]],
    textposition="outside",
    textfont=dict(color=TEXT_COLOR, size=12),
))
fig3.update_layout(
    title="What Drives Weight Loss? (Random Forest Feature Importance)",
    paper_bgcolor=DARK_BG,
    plot_bgcolor=PANEL_BG,
    font=dict(color=TEXT_COLOR, family="'Courier New', monospace"),
    title_font=dict(size=18, color=ACCENT_GOLD),
    xaxis=dict(
        title="Importance Score",
        gridcolor=GRID_COLOR,
        range=[0, importance_df["Importance"].max() * 1.25],
    ),
    yaxis=dict(gridcolor=GRID_COLOR),
    margin=dict(t=60, b=50, l=160, r=80),
)

# ── Assemble into a single HTML dashboard ───────────────────
def fig_to_div(fig, include_plotlyjs=False):
    """Convert a Plotly figure to an HTML div string."""
    return fig.to_html(
        full_html=False,
        include_plotlyjs=include_plotlyjs,
        div_id=None,
        config={"displayModeBar": True, "responsive": True},
    )

html_template = """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>Fasting Vitals Analyzer &amp; Predictor</title>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
  <link href="https://fonts.googleapis.com/css2?family=Share+Tech+Mono&family=Rajdhani:wght@400;600;700&display=swap" rel="stylesheet"/>
  <style>
    *, *::before, *::after {{ box-sizing: border-box; margin: 0; padding: 0; }}

    :root {{
      --bg        : #0d1117;
      --panel     : #161b22;
      --border    : #21262d;
      --teal      : #00c9a7;
      --pink      : #ff6b9d;
      --gold      : #ffd166;
      --text      : #e6edf3;
      --muted     : #8b949e;
    }}

    body {{
      background : var(--bg);
      color      : var(--text);
      font-family: 'Rajdhani', sans-serif;
      min-height : 100vh;
    }}

    /* ── Header ── */
    .header {{
      display         : flex;
      align-items     : center;
      justify-content : space-between;
      padding         : 28px 40px 20px;
      border-bottom   : 1px solid var(--border);
      background      : linear-gradient(90deg, #0d1117 0%, #0f1f1a 100%);
    }}
    .header-left h1 {{
      font-size     : 2rem;
      font-weight   : 700;
      letter-spacing: 2px;
      color         : var(--teal);
      text-transform: uppercase;
    }}
    .header-left p {{
      font-family: 'Share Tech Mono', monospace;
      color      : var(--muted);
      font-size  : 0.85rem;
      margin-top : 4px;
    }}
    .badge {{
      background    : var(--panel);
      border        : 1px solid var(--teal);
      color         : var(--teal);
      font-family   : 'Share Tech Mono', monospace;
      font-size     : 0.75rem;
      padding       : 6px 14px;
      border-radius : 4px;
      letter-spacing: 1px;
    }}

    /* ── KPI strip ── */
    .kpi-strip {{
      display        : flex;
      gap            : 1px;
      background     : var(--border);
      border-bottom  : 1px solid var(--border);
    }}
    .kpi {{
      flex        : 1;
      background  : var(--panel);
      padding     : 18px 24px;
      text-align  : center;
    }}
    .kpi-value {{
      font-size     : 1.9rem;
      font-weight   : 700;
      letter-spacing: 1px;
    }}
    .kpi-label {{
      font-family: 'Share Tech Mono', monospace;
      font-size  : 0.72rem;
      color      : var(--muted);
      margin-top : 2px;
      text-transform: uppercase;
    }}

    /* ── Chart grid ── */
    .grid {{
      display              : grid;
      grid-template-columns: 1fr 1fr;
      grid-template-rows   : auto auto;
      gap                  : 1px;
      background           : var(--border);
      padding-top          : 1px;
    }}
    .chart-card {{
      background : var(--panel);
      padding    : 10px;
      position   : relative;
    }}
    .chart-card.full-width {{
      grid-column: 1 / -1;
    }}
    .chart-card::before {{
      content  : '';
      position : absolute;
      top      : 0; left: 0; right: 0;
      height   : 2px;
      background: linear-gradient(90deg, var(--teal), var(--pink));
    }}

    /* ── Footer ── */
    .footer {{
      text-align : center;
      padding    : 20px;
      font-family: 'Share Tech Mono', monospace;
      font-size  : 0.72rem;
      color      : var(--muted);
      border-top : 1px solid var(--border);
    }}
  </style>
</head>
<body>

  <!-- ── HEADER ── -->
  <header class="header">
    <div class="header-left">
      <h1>⚡ Fasting Vitals Analyzer</h1>
      <p>Medical Informatics Portfolio · Synthetic Dataset · N = 500 patients</p>
    </div>
    <span class="badge">RandomForest · R² {r2:.4f} · RMSE {rmse:.4f} kg</span>
  </header>

  <!-- ── KPI STRIP ── -->
  <div class="kpi-strip">
    <div class="kpi">
      <div class="kpi-value" style="color:var(--teal)">{avg_wl:.2f} kg</div>
      <div class="kpi-label">Avg Weight Loss</div>
    </div>
    <div class="kpi">
      <div class="kpi-value" style="color:var(--pink)">{avg_bs:.1f} mg/dL</div>
      <div class="kpi-label">Avg Blood Sugar Drop</div>
    </div>
    <div class="kpi">
      <div class="kpi-value" style="color:var(--gold)">{avg_days:.1f} days</div>
      <div class="kpi-label">Avg Days Fasted</div>
    </div>
    <div class="kpi">
      <div class="kpi-value" style="color:#a78bfa">{avg_hrs:.1f} hrs</div>
      <div class="kpi-label">Avg Daily Fasting Window</div>
    </div>
  </div>

  <!-- ── CHARTS ── -->
  <div class="grid">
    <div class="chart-card full-width">
      {fig1_div}
    </div>
    <div class="chart-card">
      {fig2_div}
    </div>
    <div class="chart-card">
      {fig3_div}
    </div>
  </div>

  <footer class="footer">
    Generated by Fasting Vitals Analyzer · Medical Informatics Portfolio · Synthetic data for demonstration only
  </footer>

</body>
</html>
"""

# Render HTML
dashboard_html = html_template.format(
    r2       = r2,
    rmse     = rmse,
    avg_wl   = df["Weight_Loss_kg"].mean(),
    avg_bs   = df["Blood_Sugar_Drop"].mean(),
    avg_days = df["Days_Fasted"].mean(),
    avg_hrs  = df["Avg_Fasting_Hours"].mean(),
    fig1_div = fig_to_div(fig1, include_plotlyjs=False),
    fig2_div = fig_to_div(fig2, include_plotlyjs=False),
    fig3_div = fig_to_div(fig3, include_plotlyjs=False),
)

# Save dashboard
OUTPUT_FILE = "Fasting_Health_Dashboard.html"
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    f.write(dashboard_html)

print("\n" + "═" * 55)
print(f"  ✅  Dashboard saved → {OUTPUT_FILE}")
print("═" * 55)

# ── Google Colab: auto-download link ────────────────────────
try:
    from google.colab import files
    files.download(OUTPUT_FILE)
    print("  📥  Download triggered in Colab.")
except ImportError:
    print("  ℹ️   Not running in Colab — open the saved HTML file directly.")

# ── Optional: inline display inside Colab notebook ──────────
try:
    from IPython.display import IFrame, display
    display(IFrame(src=OUTPUT_FILE, width="100%", height="900px"))
except Exception:
    pass